# Data Cleaning Script

**Dataset:** `./ecommerce_user_segmentation.csv`

## Steps:
1. Load & inspect
2. Clean (missing values, dates, strings, columns)
3. Handle duplicates
4. Detect outliers (IQR) – DO NOT REMOVE
5. Final checks & summary
6. Export cleaned dataset


## 1. IMPORT LIBRARIES & LOAD DATA


In [ ]:
import pandas as pd
import numpy as np

# 1.1 Load the dataset
file_path = "../Data/ecommerce_user_segmentation.csv"
df = pd.read_csv(file_path)

# 1.2 Display first 10 rows
print("\n===== First 10 rows of the dataset =====")
print(df.head(10))

# 1.3 Display shape (rows, columns)
print("\n===== Dataset Shape (rows, columns) =====")
print(df.shape)

# 1.4 Display data types per column
print("\n===== Column Data Types =====")
print(df.dtypes)

# 1.5 Display missing values count per column
print("\n===== Missing Values per Column =====")
print(df.isnull().sum())


## 2. CLEAN THE DATASET


In [ ]:
# 2.1 Fill missing values in NUMERICAL columns with MEAN
numeric_cols = df.select_dtypes(include=[np.number]).columns
print("\n===== Numerical Columns (for mean imputation) =====")
print(list(numeric_cols))

for col in numeric_cols:
    mean_value = df[col].mean()
    df[col].fillna(mean_value, inplace=True)

# 2.2 Fill missing values in CATEGORICAL columns with MODE
# Include 'object', 'category', and pandas "string" dtype as categorical
categorical_cols = df.select_dtypes(include=['object', 'category', 'string']).columns
print("\n===== Categorical Columns (for mode imputation) =====")
print(list(categorical_cols))

for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        mode_value = df[col].mode(dropna=True)
        if not mode_value.empty:
            df[col].fillna(mode_value.iloc[0], inplace=True)

# 2.3 Convert date/time columns (if any) into datetime format
# Here we infer date/time columns based on column names containing common keywords
date_like_keywords = ["date", "time", "datetime", "timestamp"]
date_cols = [col for col in df.columns
             if any(kw in col.lower() for kw in date_like_keywords)]

print("\n===== Columns considered as Date/Time =====")
print(date_cols)

for col in date_cols:
    # errors='coerce' will turn invalid dates into NaT
    df[col] = pd.to_datetime(df[col], errors='coerce')

# 2.4 Strip whitespace from all string/text (object) columns
for col in categorical_cols:
    # Only apply strip if the column is of object/string type
    if df[col].dtype == 'object':
        df[col] = df[col].astype(str).str.strip()

# 2.5 Convert all column names to lowercase and replace spaces with underscores
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

print("\n===== Column Names After Standardization =====")
print(list(df.columns))


## 3. HANDLE DUPLICATE ENTRIES


In [ ]:
# 3.1 Count duplicate rows
duplicate_count = df.duplicated().sum()
print("\n===== Duplicate Rows Count =====")
print(f"Number of duplicate rows: {duplicate_count}")

# 3.2 Remove duplicates
before_rows = df.shape[0]
df = df.drop_duplicates()
after_rows = df.shape[0]

# 3.3 Print how many rows were removed
removed_rows = before_rows - after_rows
print("\n===== Duplicate Removal Summary =====")
print(f"Rows before removing duplicates: {before_rows}")
print(f"Rows after removing duplicates:  {after_rows}")
print(f"Total duplicate rows removed:    {removed_rows}")


## 4. DETECT OUTLIERS USING IQR (DO NOT REMOVE)


In [ ]:
print("\n===== Outlier Detection using IQR (per numerical column) =====")
numeric_cols = df.select_dtypes(include=[np.number]).columns  # re-evaluate after cleaning
outlier_summary = {}

for col in numeric_cols:
    col_data = df[col].dropna()
    if col_data.empty:
        outlier_summary[col] = 0
        continue

    Q1 = col_data.quantile(0.25)
    Q3 = col_data.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 + (1.5 * -IQR)  # same as Q1 - 1.5*IQR
    upper_bound = Q3 + (1.5 * IQR)

    outliers = col_data[(col_data < lower_bound) | (col_data > upper_bound)]
    outlier_count = outliers.shape[0]
    outlier_summary[col] = outlier_count

    print(f"Column '{col}': {outlier_count} outliers (IQR method)")

# Short interpretation for outliers
print("\n===== Outlier Interpretation (High-level) =====")
for col, count in outlier_summary.items():
    if count > 0:
        print(f"- '{col}' has {count} potential outliers. "
              f"Business team may decide later whether these are real extreme cases "
              f"(e.g., very high spenders) or data errors.")
    else:
        print(f"- '{col}' has no strong outliers detected by IQR.")


## 5. FINAL CHECKS AFTER CLEANING


In [ ]:
# 5.1 Print missing values again to confirm cleanup
print("\n===== Missing Values per Column AFTER Cleaning =====")
print(df.isnull().sum())

# 5.2 Print final dataset shape
print("\n===== Final Dataset Shape (rows, columns) =====")
print(df.shape)

# 5.3 Beginner-friendly summary of what was cleaned and why
print("\n===== Cleaning Summary (Beginner-Friendly) =====")
summary_points = [
    "1. Filled empty values in number columns with the average (mean) so that calculations work smoothly.",
    "2. Filled empty values in text/category columns with the most common value (mode) to keep categories consistent.",
    "3. Converted any date/time-like columns into proper datetime format so we can do time-based analysis later.",
    "4. Removed extra spaces from text fields to avoid treating 'User ' and 'User' as different values.",
    "5. Standardized column names to lowercase_with_underscores, making them easier to use in code.",
    "6. Detected and removed duplicate rows so each user/record is counted only once.",
    "7. Identified potential outliers in numeric columns using the IQR method, but did NOT remove them yet.",
    "8. Verified that there are no remaining missing values and checked the final size of the cleaned dataset."
]

for point in summary_points:
    print(point)


## 6. EXPORT THE CLEANED DATASET


In [ ]:
output_file = "../Data/cleaned_ecommerce_dataset.csv"
df.to_csv(output_file, index=False)

print(f"\n===== Export Complete =====")
print(f"Cleaned dataset saved as: {output_file} (index=False)")
